# Revenue Forecasting (Regularized Linear Regression)

## Load libraries and data

In [1]:
import io
import requests
import numpy as np
import pandas as pd
from pandas.plotting import scatter_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import (LinearRegression, Ridge,
                                  Lasso, ElasticNet)
from sklearn.metrics import mean_squared_error, r2_score
import matplotlib as mpl
import matplotlib.pyplot as plt
import seaborn as sns

# Make this notebook's output stable across runs
np.random.seed(100)

# Plot formatting
%matplotlib inline
sns.set()
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Do not use scientific notation in pandas tables
pd.options.display.float_format = '{:,.4f}'.format

### Data Description

When retailers close stores, they usually conduct going-out-of-business (GOB) sales.

In the data set, each row represents a store's going out of business sale.

| Attribute             | Definition                                                                |
| --------------------- | ------------------------------------------------------------------------- |
| StoreType             | Store's retail segment (bookstores or household goods)                    |
| LiquidationRevenue    | Revenue from the GOB sale                                                 |
| InventoryAtTicket     | Retail value of inventory held at the start of the GOB sale               |
| LastYearRevenue       | Store's revenue over the GOB sale dates during the prior year             |                      
| MedianHouseholdIncome | Median household income in the store's ZIP code (from U.S. Census Bureau) |
| MedianAge             | Median age in the store's ZIP code (from U.S. Census Bureau)              |


In [2]:
url = 'https://raw.githubusercontent.com/natecraig/aiml/main/Data/closings.csv'
download = requests.get(url).content
df = pd.read_csv(io.StringIO(download.decode('utf-8')))
df.head(-5)

,StoreType,LiquidationRevenue,InventoryAtTicket,LastYearRevenue,MedianHouseholdIncome,MedianAge
0,Bookstore,"1,214,776.2000","2,195,069.8000","465,237.3000",51290,37.8000
1,Bookstore,"1,811,896.9000","3,152,301.9000","897,704.5000",46702,33.4000
2,Bookstore,"1,157,614.7000","2,229,837.5000","477,804.8000",86804,41.7000
3,Bookstore,"2,037,136.2000","3,857,466.8000","1,036,097.7000",83544,40.3000
4,Bookstore,"1,326,821.1000","2,345,263.8000","612,793.5000",23508,31.9000
...,...,...,...,...,...,...
553,Household Goods,"1,823,324.8000","2,959,365.5300","936,848.3300",56052,32.1000
554,Household Goods,"1,701,298.5000","2,244,323.7300","1,361,859.4000",42424,35.8000
555,Household Goods,"1,335,948.9000","2,921,797.7800","699,678.4300",46227,30.4000
556,Household Goods,"1,923,520.5000","2,516,588.0300","1,238,979.5000",55243,34.7000


## Model

In [3]:
# Transform features
df['Bookstore'] = df['StoreType'].apply(lambda x: 1 if x == 'Bookstore' else 0)
df['BookstoreXInventoryAtTicket'] = df['Bookstore'] * df['InventoryAtTicket']

# Add square terms
numvars = ['InventoryAtTicket', 'MedianHouseholdIncome']
for v in numvars:
    df[v + 'Sq'] = df[v]**2
    
# Set the target, y, and features, X
y = df['LiquidationRevenue']

feature_names = ['Bookstore', 'BookstoreXInventoryAtTicket',
                 'InventoryAtTicket', 'InventoryAtTicketSq',
                 'MedianHouseholdIncome', 'MedianHouseholdIncomeSq']
X = df[feature_names]

# Scale data
scaler = StandardScaler()
X_scale = scaler.fit_transform(X)

# Split data into training and test sets

# Note that we are using a large test size (75%) to emphasize the
# challenge of generalization
(X_train, X_test,
 y_train, y_test) = train_test_split(X_scale, y, test_size=0.75)

In [4]:
# Fit a linear regresion
linreg = LinearRegression()
linreg.fit(X_train, y_train)
y_train_pred = linreg.predict(X_train)
y_pred = linreg.predict(X_test)

In [5]:
# Retain coefficients and performance
df_regs = pd.DataFrame(columns=(['Intercept'] + feature_names
                                + ['Train R2', 'Test R2']))
df_regs.loc['Linear'] = ([linreg.intercept_] + list(linreg.coef_)
                         + [r2_score(y_train, y_train_pred),
                            r2_score(y_test, y_pred)])

In [6]:
# Set regularization parameter for ridge, lasso, and elasticnet
alpha=2
l1_ratio=0.5

## Ridge Regression

In [7]:
ridge = Ridge(alpha=alpha)
ridge.fit(X_train, y_train)
y_train_pred = ridge.predict(X_train)
y_pred = ridge.predict(X_test)

df_regs.loc['Ridge'] = ([ridge.intercept_] + list(ridge.coef_)
                         + [r2_score(y_train, y_train_pred),
                            r2_score(y_test, y_pred)])

## LASSO Regression

In [8]:
lasso = Lasso(alpha=alpha, max_iter=100000)
lasso.fit(X_train, y_train)
y_train_pred = lasso.predict(X_train)
y_pred = lasso.predict(X_test)

df_regs.loc['Lasso'] = ([lasso.intercept_] + list(lasso.coef_)
                         + [r2_score(y_train, y_train_pred),
                            r2_score(y_test, y_pred)])

## Elastic Net Regression

In [9]:
elastic = ElasticNet(alpha=alpha, l1_ratio=l1_ratio)
elastic.fit(X_train, y_train)
y_train_pred = elastic.predict(X_train)
y_pred = elastic.predict(X_test)

df_regs.loc['ElasticNet'] = ([elastic.intercept_] + list(elastic.coef_)
                             + [r2_score(y_train, y_train_pred),
                                r2_score(y_test, y_pred)])

## Compare Regularization Methods

In [10]:
df_regs

,Intercept,Bookstore,BookstoreXInventoryAtTicket,InventoryAtTicket,InventoryAtTicketSq,MedianHouseholdIncome,MedianHouseholdIncomeSq,Train R2,Test R2
Linear,"1,756,243.7299","518,455.6710","-657,002.4275","1,092,316.5678","-565,560.1025","234,845.9518","-260,800.7061",0.8350,0.7720
Ridge,"1,772,018.0259","-94,260.0153","-140,666.8102","266,904.5789","124,329.4978","119,850.1682","-126,339.8095",0.8278,0.8222
Lasso,"1,756,280.5748","516,904.7844","-655,692.2300","1,090,257.6630","-563,842.8231","234,709.4959","-260,635.5717",0.8350,0.7722
ElasticNet,"1,764,730.2758","-118,528.4487","-96,173.1549","131,436.7686","121,057.4608","13,813.8230",235.8409,0.7658,0.7498


## Exercise

Try different values for alpha and l1_ratio. How does Test R2 change?